<p style="text-align: justify">The <em>Genes</em> method from MEvoLib uses only the biological knowledge provided by the user in the sequences file, preventing any guided or prior information in the source code (apart from misspelling checkers). Thus, the method is very likely to run accurately even if the information changes or the classification criteria are modified due to new discoveries. The method collects all the available information of each sequence and it determines which parts of a set of sequences refer to the same gene using algebra of sets. This methodology works properly even though the genes might be in different loci.</p>
<p style="text-align: justify">To show the performance of the <em>Genes</em> method we are going to download all the complete human mitochondrial DNA (<em>hmtDNA</em>) sequences available at GenBank. The 31755 sequences (on 07/Jul/2016) match the following query:
<pre>
"homo sapiens"[porgn] AND mitochondrion[Filter] NOT mRNA[Filter] AND "complete genome"
</pre>
</p>

In [ ]:
from MEvoLib.Fetch.BioSeqs import BioSeqs

seq_db = BioSeqs.from_entrez(email='eg@test.com', entrez_db='nuccore',
             query='"homo sapiens"[porgn] AND mitochondrion[Filter] NOT mRNA[Filter] AND "complete genome"')
seq_db.write('hmtDNA_{}seqs.gb'.format(len(seq_db)))
print(seq_db.statistics())

<p style="text-align: justify">Before running all the tests, we execute the <em>Genes</em> method with all the sequences in its default configuration to analyze the log file to detect possible errors in the biological information. Three sequences have been removed from the downloaded set: <strong>i)</strong> KP702293.1 has the gene ND3's product as "NADH dehydrogenase subunit 2"; <strong>ii)</strong> FR695060.1 has the CDS "ATP synthase 6" identified as ATP8; and, <strong>iii)</strong> DQ862537.1 has the gene ATP6's product as "cytochrome c oxidase subunit III".</p>

In [ ]:
from MEvoLib import Cluster

gene_dict = Cluster.get_subsets('genes', 'hmtDNA_{}seqs.gb'.format(len(seq_db)), 'gb',
                ['CDS', 'rRNA', 'tRNA'], None, None, 'hmtDNA_{}seqs.log'.format(len(seq_db)))
for record_id in ['KP702293.1', 'FR695060.1', 'DQ862537.1']
    del seq_db.data[record_id]
seq_db.write('hmtDNA_{}seqs.gb'.format(len(seq_db)))
print(seq_db.statistics())

<p style="text-align: justify">We have also created three subsets with the first 100, 1000 and 10000 sequences of the curated dataset to perform an additional scalability test.</p>

In [ ]:
from Bio import SeqIO

subset = []
for i, record in enumerate(SeqIO.parse('hmtDNA_{}seqs.gb'.format(len(seq_db)), 'gb')) :
    s10000.append(record)
    if ( i == 10000 ) :
        break
SeqIO.write(subset[0:100], 'hmtDNA_100seqs.gb', 'gb')
SeqIO.write(subset[0:1000], 'hmtDNA_1000seqs.gb', 'gb')
SeqIO.write(subset, 'hmtDNA_10000seqs.gb', 'gb')

<p style="text-align: justify">The classical approach to withdraw different genes from a set of sequences only extracts information from <em>gene</em> or <em>product</em> qualifiers from the features provided by GenBank. To demonstrate the improvement that the <em>Genes</em> method offers in comparison to the classical approach we are going to test the three options: <em>all</em> qualifiers, only the <em>gene</em> qualifier and only the <em>product</em> qualifier.</p>
<p style="text-align: justify">We have focused the outputs on the time cost and the recovery rate of the coding DNA sequence (<em>CDS</em>), ribosomal RNA (<em>rRNA</em>) and transfer RNA (<em>tRNA</em>) features for each configuration. The hmtDNA contains 13 CDS, 2 rRNA and 22 tRNA genes. The tRNAs for Leucine and Serine have two loci instead of one, located at different sites and strands of the hmtDNA. In just a few cases these two loci are differentiated by a "1" or "2" added to their qualifiers. Thus, the tRNA feature is never recovered completely.</p>

In [ ]:
import time

for num_seqs in [100, 1000, 10000, int(len(seq_db))] :
    filename = 'hmtDNA_{}seqs'.format(num_seqs)
    start = time.time()
    gene_dict = Cluster.get_subsets('genes', '{}.gb'.format(filename), 'gb',
                    ['CDS', 'rRNA', 'tRNA'], None, None, '{}.log'.format(filename))
    end = time.time()
    print('time cost: {:.4f} seconds'.format(end - start))
    for key, value in sorted(gene_dict.items()) :
        if ( len(value) > 0 ) :
            print(key)

<p style="text-align: justify">To perform the classical approach, we have modified the default behavior of the <em>Genes</em> method: instead of collecting the information of all the qualifiers of each feature, it only takes into account the <em>gene</em> or <em>product</em> qualifiers, respectively.</p>

In [ ]:
from MEvoLib.Cluster import Genes

Genes._FEAT_QUAL_DICT['CDS'] = ['gene']
Genes._FEAT_QUAL_DICT['rRNA'] = ['gene']
Genes._FEAT_QUAL_DICT['tRNA'] = ['gene']
for num_seqs in [100, 1000, 10000, int(len(seq_db))] :
    filename = 'hmtDNA_{}seqs'.format(num_seqs)
    start = time.time()
    gene_dict = Cluster.get_subsets('genes', '{}.gb'.format(filename), 'gb',
                    ['CDS', 'rRNA', 'tRNA'], None, None, '{}.log'.format(filename))
    end = time.time()
    print('time cost: {:.4f} seconds'.format(end - start))
    for key, value in sorted(gene_dict.items()) :
        if ( len(value) > 0 ) :
            print(key)

In [ ]:
Genes._FEAT_QUAL_DICT['CDS'] = ['product']
Genes._FEAT_QUAL_DICT['rRNA'] = ['product']
Genes._FEAT_QUAL_DICT['tRNA'] = ['product']
for num_seqs in [100, 1000, 10000, int(len(seq_db))] :
    filename = 'hmtDNA_{}seqs'.format(num_seqs)
    start = time.time()
    gene_dict = Cluster.get_subsets('genes', '{}.gb'.format(filename), 'gb',
                    ['CDS', 'rRNA', 'tRNA'], None, None, '{}.log'.format(filename))
    end = time.time()
    print('time cost: {:.4f} seconds'.format(end - start))
    for key, value in sorted(gene_dict.items()) :
        if ( len(value) > 0 ) :
            print(key)